<a href="https://colab.research.google.com/github/cocorini/KT_AIVLE/blob/main/6%EC%B0%A8_%EB%AF%B8%ED%94%84_by_KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### KoBERT 관련 모듈 호출

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-0wrfi003/kobert-tokenizer_a5962048114e4b95ab6d416776809217
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-0wrfi003/kobert-tokenizer_a5962048114e4b95ab6d416776809217
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=110eea47e2e1a3309eebe04130031c9c86ec9c134438fe4560407f3a4e374adf
  Stored in directory: /tmp/pip-ephem-wheel-cache-gjee5xxx/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
import torch
import random
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

* 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#ex : path = '/content/drive/MyDrive/project/'
path = '/content/drive/MyDrive/2023.10.30_미니프로젝트6차_실습자료 및 데이터/데이터'

저장된 데이터 읽어오기
* 저장된 .pkl 파일들을 불러옵니다.
* 불러 온 후에는 shape를 확인해 봅시다.

In [ ]:
train = joblib.load(os.path.join(path, 'train.pkl'))
test = joblib.load(os.path.join(path, 'test.pkl'))

# okt로 질문을 형태소 분석한 결과가 데이터 프레임에 담긴 상태
clean_train = joblib.load(os.path.join(path, 'clean_train.pkl'))
clean_test = joblib.load(os.path.join(path, 'clean_test.pkl'))

### KoBERT

* 형식에 맞게 전처리하기
    - input 데이터의 형식 : ['text', 'label'] 형식으로 들어가야함.

사실 label 이 str형으로만 해야하는지는 확신이 없다.

In [ ]:
train.intent = train.intent-1
test.intent = test.intent-1

In [ ]:
train_data = []
test_data = []
for q, label in zip(train['Q'], train['intent']) :
    data = []
    data.append(q)
    data.append(str(label))

    train_data.append(data)

for q, label in zip(test['Q'], test['intent']) :
    data = []
    data.append(q)
    data.append(str(label))

    test_data.append(data)

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

# # CPU 사용
# device = torch.device("cpu")

#BERT 모델, Vocabulary 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 35
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tok = tokenizer.tokenize

data_train = BERTDataset(train_data, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(test_data, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=53,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
KoBERT = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in KoBERT.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in KoBERT.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    KoBERT.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = KoBERT(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(KoBERT.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    KoBERT.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = KoBERT(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/19 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.1815905570983887 train acc 0.953125
epoch 1 train acc 0.9819078947368421


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 1 test acc 0.8318452380952381


  0%|          | 0/19 [00:00<?, ?it/s]

...

epoch 2 test acc 0.8199404761904762


  0%|          | 0/19 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 1.1271358728408813 train acc 0.96875
epoch 3 train acc 0.9893092105263158


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 3 test acc 0.8318452380952381


In [ ]:
test_acc = 0.0
KoBERT.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = KoBERT(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
print("epoch {} test acc {}".format(1, test_acc / (batch_id+1)))

  0%|          | 0/2 [00:00<?, ?it/s]

epoch 1 test acc 0.8318452380952381


In [ ]:
def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환

    KoBERT.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = KoBERT(token_ids, valid_length, segment_ids)

        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()
            intent = np.argmax(logits)
            ans = random.choice(train.loc[train['intent']==intent, 'A'].unique())

        print(ans)

    return intent

* 모델 저장하기 & 불러오기

In [ ]:
#학습된 KOBERT 모델 kobert로 저장하기
joblib.dump(KoBERT, 'kobert.pkl')

['kobert.pkl']

In [ ]:
#학습된 KOBERT모델 불러오기
KoBERT = joblib.load('/content/drive/MyDrive/2023.10.30_미니프로젝트6차_실습자료 및 데이터/데이터/kobert.pkl')

* 챗봇 구축

In [ ]:
input = '아직 대학 졸업예정인데, 지원 가능한가요?'

In [ ]:
# 분류 모델을 이용하여 intent 예측
predict(input)

KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자 중 만 34세 이하 미취업자를 대상으로 하는 교육입니다.
단, 모집시점에 만 35세여도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다.
또한 전공의 종류와는 무관 합니다.


In [ ]:
input2 = '비전공자도 가능한가요?'
predict(input2)

비전공자도 SW개발에 관심과 열정이 있으신 분들은 충분히 따라 갈 수 있습니다. 이 경우 데이터 사이언스 기반으로 산업 현장에 AI를 접목하고 DX를 이끄는 DX컨설턴트 Track을 추천 드립니다. SW기초 지식 및 코딩역량을 보유하고 있고 개발자로 성장하고 싶은 분께는 AI개발자 Track을 추천 드립니다. 파이썬 프로그래밍 사용법과 AI 개념에 대한 기초 이해를 공부하고 오시면 도움이 되실 것 같습니다.


In [ ]:
input3 = '배고프다'
predict(input3)

힘들겠어요.


In [ ]:
def get_answer1(question):
    intent = predict(question)
    return

In [ ]:
get_answer1('교육 듣다가 중도포기 가능한가요?')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


교육 등록신청을 하지 않거나 교육 일정 시작 전에 교육 등록을 취소해도 지원자격에 해당하는 경우 다시 지원이 가능합니다.
다만, 교육 과정 확정자신고일(교육시작일로부터 7일)이 지난 후 중도 퇴교를 할 경우 과정을 수강한 것으로 간주되며, 향후 K-Digital Training (K-DT) 과정에 재지원은 가능하나 무료 수강은 어렵습니다.


33

In [ ]:
cnt=0
for _, intent in enumerate(test_data):
    print("질문:", intent[0], "\n대답:", end=' ')
    intent_pred = get_answer1(intent[0])
    if int(intent[1])==intent_pred:
        cnt+=1
    print()

질문: 노트북이 작동이 안되네 
대답: as 맡기세요.

질문: 핸드폰 맛이 갔어 
대답: 교육기간 동안 노트북을 무료로 지원하며, 교육에 필요한 필수 프로그램이 설치되어 있으므로, 제공한 노트북을 사용해 주시기 바랍니다.

원활한 실습 환경을 위해 아래 명시된 사양 이상을 갖춘 노트북을 제공합니다. 
- CPU : I5 / RAM : 8GB / 여유저장공간 : 40GB

질문: 내 의지는 상관없나봐 
대답: KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자 중 만 34세 이하 미취업자를 대상으로 하는 교육입니다.
단, 모집시점에 만 35세여도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다.
또한 전공의 종류와는 무관 합니다.

질문: 올해 취업하길 
대답: 가장 중요한 거예요.

질문: 추워죽겠다 
대답: 감기 조심하세요.

질문: 바람 많이 부네 
대답: 감기 조심하세요.

질문: 나들이를 가볼까 
대답: 같이 가보세요.

질문: 바다 가자고 하면 갈까? 
대답: 같이 가자고 말해보세요

질문: mmm 
대답: 교육기간 동안 노트북을 무료로 지원하며, 교육에 필요한 필수 프로그램이 설치되어 있으므로, 제공한 노트북을 사용해 주시기 바랍니다.

원활한 실습 환경을 위해 아래 명시된 사양 이상을 갖춘 노트북을 제공합니다. 
- CPU : I5 / RAM : 8GB / 여유저장공간 : 40GB

질문: 불면증 온 거 같아. 
대답: 고민이 많은가 봐요.

질문: 공부 시작해도 될까 
대답: 공부가 최우선이죠.

질문: 새로운 공부 시작해도 될까 
대답: 공부는 끝이 없죠.

질문: 친구가 내 뒷담 깠어 
대답: 너무 신경쓰지마세요.

질문: 나를 친구로 생각 안했나봐 
대답: 그런 친구 말고 제대로 된 친구 사귀세요.

질문: 단발로 자를까 
대답: 힘들겠네요.

질문: 계속 열심히 공부하는데 집중이 잘 안되네 
대답: 비전공자도 SW개발에 관심과 열정이 있으신 분들은 충분히 따라 갈 수 있습니다.  
이 경우 데이터 사이언스 기반으로 산업 현장에 

In [ ]:
print(cnt/len(test_data))

0.8018867924528302


In [ ]:
get_answer1("수료 직후 취업 가능?")

KT 에이블스쿨은 미래인재육성 + 청년 IT/SW/DX 분야 일자리 창출을 위해서 KT 와 정부가 함께 하는 프로그램입니다.  
에이블러 여러분들의 취업기회를 위해 Job Searching 세션, 취업컨설팅, 잡페어 등 여러 프로그램들을 운영하고 있습니다.

KT 직접채용은 그 일환 중 하나이며, KT에 합격한 1기 수료생의 경우 해당지역에 배치되었습니다.  
KT 외에 KT 그룹사, 주요 협력사, AI원팀 기업, 스타트업 등의 채용은 지역 구분 없이 진행됩니다. 
채용 상황에 따라 변경될 수 있어 교육 수료 후 채용 연계 시 상세 안내 드리겠습니다. 

KT 그룹, AI 원팀 기업/스타트업 100 등 다양한 기업에서 채용 연계할 예정으로 기업마다 인재상 및 반영 요소가 다르기 때문에 중점 요소에 대해 안내드리기 어려운 점 양해 부탁드립니다. 채용에 대한 부분은 교육을 참여하시면 추후 관련 부서를 통해 안내드릴 예정입니다.


In [ ]:
get_answer1("취업하고 싶어요")

저도요.


In [ ]:
get_answer1("이거 하고 취업 못하면 책임 져요?")

KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다.
미취업자의 기준은 아래와 같습니다.
1) 기간의 정함이 있는 근로인 경우
2) 고용보험에 미가입한 경우
3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우
단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다. 

프리랜서 계약의 경우, 고용보험에 가입하지 않은 경우 미취업자로 보지만, 개인별 상황이 다를 수 있어서 관할 고용센터에 문의하여 확인하시기 바랍니다. 
또한 현재 군 복무 중인 상태여도 지원 가능하며, 교육 시작 전일까지 병역 의무를 마쳐야 교육 수강이 가능합니다. 

퇴사 후 고용보험 상실이 완료되어 실업급여를 수급 중인 경우, 훈련 수강에는 문제가 없습니다.
다만, 실업급여와 훈련장려금은 중복 수급이 불가하여 실업급여 지급이 종료된 후 훈련장려금 지급이 개시됩니다.
